# Exceso de mortalidad en Chile
> Exceso de mortalidad en Chile

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/fallecimientos.png

In [1]:
#hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import datetime

In [106]:
#hide
from IPython.display import display, Markdown, display_html, HTML

In [2]:
#hide
deaths_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto32/Defunciones.csv")

# Exceso de mortalidad en Chile

In [3]:
#hide
deaths = deaths_raw.drop(columns=["Region", "Codigo region", "Comuna", "Codigo comuna"]).sum()

In [4]:
#hide
# sort rows by date/index
deaths.index = pd.to_datetime(deaths.index)
deaths = deaths.sort_index()
last_day = deaths.index[-1]
last_day.strftime("%Y-%m-%d")

'2020-06-04'

In [23]:
#hide
#Give two days so that they have time to add inscriptions
_, current_week, _ = (last_day-pd.DateOffset(2)).isocalendar()
current_week

23

In [6]:
#hide
deaths = (deaths
          .reset_index()
          .rename(columns={"index": "fecha", 0: "fallecidos"})
          )

In [7]:
#hide
def get_isoyear_isoweek(row):
    isoyear, isoweek, _ = row["fecha"].isocalendar()
    return pd.Series({"año": isoyear, "semana": isoweek})

In [8]:
#hide
deaths[["año", "semana"]] = deaths.apply(get_isoyear_isoweek, axis="columns")

In [9]:
#hide
deaths_year_week = (deaths
                    .drop(columns=["fecha"])
                    .groupby(["año", "semana"])
                    .sum()
                    ["fallecidos"]
                    .unstack()
                    .astype("Float16")
                    )

In [10]:
#hide
deaths_year_week = deaths_year_week.iloc[1:,:-1]

In [11]:
#hide
deaths_year_week

semana,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
año,,,,,,,,,,,,,,,,,,,,,
2010,1786.0,1672.0,1775.0,1689.0,1687.0,1645.0,1685.0,1641.0,1865.0,1852.0,...,1906.0,1783.0,1722.0,1703.0,1738.0,1617.0,1810.0,1663.0,1725.0,1789.0
2011,1814.0,1718.0,1743.0,1795.0,1703.0,1618.0,1562.0,1657.0,1679.0,1542.0,...,1837.0,1673.0,1751.0,1714.0,1743.0,1750.0,1720.0,1735.0,1654.0,1911.0
2012,1937.0,1736.0,1730.0,1717.0,1731.0,1718.0,1669.0,1654.0,1697.0,1767.0,...,1923.0,1670.0,2019.0,1871.0,1728.0,1799.0,1812.0,1657.0,1759.0,1853.0
2013,1866.0,1855.0,1756.0,1817.0,1729.0,1867.0,1698.0,1643.0,1822.0,1708.0,...,1848.0,1619.0,2038.0,1904.0,1890.0,1756.0,1880.0,1705.0,1917.0,1855.0
2014,1841.0,1949.0,1852.0,1829.0,1744.0,1683.0,1756.0,1756.0,1701.0,1691.0,...,1934.0,1716.0,2052.0,1967.0,1855.0,1846.0,1910.0,1745.0,1827.0,1822.0
2015,1796.0,1888.0,1852.0,1792.0,1812.0,1875.0,1829.0,1824.0,1844.0,1703.0,...,1992.0,1619.0,2028.0,2236.0,1949.0,1863.0,1960.0,1915.0,1879.0,1768.0
2016,2116.0,1858.0,1932.0,1863.0,1910.0,1749.0,1724.0,1836.0,1854.0,1765.0,...,2088.0,1761.0,1927.0,1800.0,1854.0,1874.0,1811.0,1891.0,1788.0,1825.0
2017,1874.0,1874.0,1958.0,2110.0,1940.0,1815.0,1854.0,1893.0,1830.0,1731.0,...,1800.0,1930.0,1973.0,1816.0,1943.0,1907.0,1737.0,2006.0,2023.0,1936.0
2018,1861.0,1865.0,1778.0,1779.0,1860.0,1885.0,1868.0,1818.0,1780.0,1904.0,...,2064.0,1761.0,2226.0,1927.0,1986.0,1906.0,1932.0,1907.0,1980.0,1892.0


In [13]:
#hide
deaths_year_week.loc[2020,current_week] = np.NaN

In [25]:
#hide
expected = deaths_year_week.loc[2015:2019].mean()

In [26]:
#hide
ci = deaths_year_week.loc[2015:2019].std(ddof=1)

In [191]:
#hide
df_expected = pd.DataFrame()
df_expected["Expected"] = expected
df_expected["lower"] = expected - ci
df_expected["upper"] = expected + ci
df_expected["lower 2"] = expected - 2*ci
df_expected["upper 2"] = expected + 2*ci

In [193]:
#hide
df_expected.head(2)

,Expected,lower,upper,lower 2,upper 2
semana,,,,,
1,2009.149791,1869.291341,2149.008242,1729.432890,2288.866693
2,1964.573910,1919.941011,2009.206809,1875.308113,2053.839708


In [194]:
#hide_input
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

base = alt.Chart(deaths_year_week.reset_index().melt("año", value_name="defunciones")).mark_line(point=True).encode(
    x = alt.X("semana:Q",scale=alt.Scale(domain=(1, 52))),
    y = alt.Y("defunciones", scale=alt.Scale(domain=(0,3500))),
    #size=alt.Size('año:N', legend=None),
    color=alt.Color('año:N', scale=alt.Scale(range=['lightgray','lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'red'], 
                                             domain=["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]))
)

alt.layer(
    base, # base line chart
    
    alt.Chart(expected.reset_index().rename(columns={0:"defunciones"})).mark_line(point=True).encode(
    x = "semana",
    y = "defunciones"
    ),
    
    alt.Chart(df_expected.reset_index()).mark_area(
        opacity=0.5
    ).encode(
        x="semana",
        y="lower:Q",
        y2="upper:Q"
    ),
    
    alt.Chart(df_expected.reset_index()).mark_area(
        opacity=0.3
    ).encode(
        x="semana",
        y="lower 2:Q",
        y2="upper 2:Q"
    ),
    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x = alt.X('semana:Q', scale=alt.Scale(domain=(1, 52)), axis=alt.Axis(title='semana'), sort=None)
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='defunciones'
    ).transform_filter(label),

    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='defunciones'
    ).transform_filter(label),
    
    data=deaths_year_week.reset_index().melt("año", value_name="defunciones")
).properties(
    title = f'Defunciones inscritas en Chile por semana (sin corregir) hasta semana {current_week-1}',
    width=600
)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19) [Registro Civil](https://www.registrocivil.cl/)

In [74]:
#hide
n_defunciones = []
for year in np.arange(2010,2020):
    n_defunciones.append(deaths_year_week.loc[year,:].astype(int).sum())

df = pd.DataFrame()
df["Año"] = np.arange(2010,2020)
df["Número de defunciones"] = n_defunciones 
df = df.set_index("Año")

In [75]:
#hide
adjustment = 365/366
df["ajustado a 365 días"] = df["Número de defunciones"].copy()
df["ajustado a 365 días"].loc[2012] *= adjustment
df["ajustado a 365 días"].loc[2016] *= adjustment
df["ajustado a 365 días"] = df["ajustado a 365 días"].astype(int)

/Users/alonsosilva/opt/anaconda3/envs/fastpages/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [76]:
#hide
growth_rate = []
for year in np.arange(2011, 2020):
    growth_rate.append(df["ajustado a 365 días"].loc[year]/df["ajustado a 365 días"].loc[year-1])
growth_rate

[0.9681922943564013,
 1.036307523778426,
 1.0153985323115078,
 1.0210208106024965,
 1.0118332173213989,
 1.0060557321137897,
 1.0290659039033834,
 0.9991015526293624,
 1.025300922673411]

In [77]:
#hide
gr_mean_iso = np.mean(growth_rate[2:])
gr_mean_iso

1.0153966673650499

In [78]:
#hide
gr_std_iso = np.std(growth_rate[2:], ddof=1)
gr_std_iso

0.010661889319897802

In [79]:
#hide
print(f"Promedio de las tasas de crecimiento anual entre 2012 y 2019: {100*(np.mean(growth_rate[2:])-1):.2f}")

Promedio de las tasas de crecimiento anual entre 2012 y 2019: 1.54


In [62]:
#hide
deaths = deaths_raw.drop(columns=["Region", "Codigo region", "Comuna", "Codigo comuna"]).sum()
n_defunciones = []
for year in np.arange(2010,2020):
    n_defunciones.append(deaths.loc[f"{year}-01-01":f"{year}-12-31"].sum())
df = pd.DataFrame()
df["Año"] = np.arange(2010,2020)
df["Número de defunciones"] = n_defunciones 
df = df.set_index("Año")

In [63]:
#hide
adjustment = 365/366
df["ajustado a 365 días"] = df["Número de defunciones"].copy()
df["ajustado a 365 días"].loc[2012] *= adjustment
df["ajustado a 365 días"].loc[2016] *= adjustment
df["ajustado a 365 días"] = df["ajustado a 365 días"].astype(int)

In [65]:
#hide
df

,Número de defunciones,ajustado a 365 días
Año,,
2010,98178,98178
2011,95105,95105
2012,99169,98898
2013,100286,100286
2014,102252,102252
2015,103710,103710
2016,104390,104104
2017,106877,106877
2018,107286,107286


In [66]:
#hide
growth_rate = []
for year in np.arange(2011, 2020):
    growth_rate.append(df["ajustado a 365 días"].loc[year]/df["ajustado a 365 días"].loc[year-1])
growth_rate

[0.9686997086923751,
 1.0398822354240052,
 1.014034661974964,
 1.0196039327523283,
 1.0142588898016665,
 1.0037990550573714,
 1.0266368247137478,
 1.0038268289716217,
 1.0237775665044833]

In [67]:
#hide
growth_rate_percentage = [f"{100*(r - 1):.2f}%" for r in growth_rate]

In [68]:
#hide
df["Variación c/r año anterior"] = ["-"]+growth_rate_percentage

In [69]:
#hide
gr_mean = np.mean(growth_rate[2:])
gr_mean

1.0151339656823117

In [70]:
#hide
gr_std = np.std(growth_rate[2:], ddof=1)
gr_std

0.008993645651094097

In [72]:
#hide
df

,Número de defunciones,ajustado a 365 días,Variación c/r año anterior
Año,,,
2010,98178,98178,-
2011,95105,95105,-3.13%
2012,99169,98898,3.99%
2013,100286,100286,1.40%
2014,102252,102252,1.96%
2015,103710,103710,1.43%
2016,104390,104104,0.38%
2017,106877,106877,2.66%
2018,107286,107286,0.38%


In [73]:
#hide
print(f"Promedio de las tasas de crecimiento anual entre 2012 y 2019: {100*(np.mean(growth_rate[2:])-1):.2f}")

Promedio de las tasas de crecimiento anual entre 2012 y 2019: 1.51


In [88]:
#hide
amended_deaths = pd.DataFrame()
for year in deaths_year_week.index[2:-1]:
    amended_deaths[year] = \
    deaths_year_week.loc[year]*df.loc[2019,"Número de defunciones"]/df.loc[year,"Número de defunciones"]*gr_mean
amended_deaths[2020] = deaths_year_week.loc[2020,:22]

In [90]:
#hide
# amended_deaths = pd.DataFrame()
# i = 8
# for year in deaths_year_week.index[2:-1]:
#     amended_deaths[year] = deaths_year_week.loc[year]*(gr_mean**i)
#     i -=1
# amended_deaths[2020] = deaths_year_week.loc[2020,:22]*(gr_mean**i)

In [91]:
#hide
amended_deaths = amended_deaths.T
amended_deaths

semana,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
2012,2181.569484,1955.190823,1948.433251,1933.791845,1949.559513,1934.918107,1879.731269,1862.837339,1911.266605,1990.104945,...,2165.801816,1880.857531,2273.922967,2107.236192,1946.180727,2026.145328,2040.786734,1866.216125,1981.094849,2086.963476
2013,2075.413972,2063.179485,1953.069097,2020.914892,1923.038991,2076.526198,1888.559981,1827.387544,2026.476022,1899.682243,...,2055.393902,1800.694116,2266.716868,2117.678566,2102.107400,1953.069097,2090.985138,1896.345564,2132.137505,2063.179485
2014,2005.452090,2123.099470,2017.434694,1992.380159,1899.787314,1833.338331,1912.859245,1912.859245,1852.946228,1842.052952,...,2106.759556,1869.286142,2235.300211,2142.707366,2020.702677,2010.898728,2080.615694,1900.876642,1990.201504,1984.754866
2015,1933.552205,2032.598309,1993.841138,1929.245853,1950.777615,2018.602664,1969.079612,1963.696672,1985.228434,1833.429513,...,2144.563471,1742.996114,2183.320642,2407.250964,2098.270183,2005.683607,2110.112652,2061.666188,2022.909017,1903.407739
2016,2258.149956,1982.817872,2061.789090,1988.153765,2038.311160,1866.495403,1839.815937,1959.339943,1978.549158,1883.570261,...,2228.268954,1879.301546,2056.453197,1920.921512,1978.549158,1999.892730,1932.660477,2018.034767,1908.115369,1947.600978
2017,1948.740127,1948.740127,2036.090271,2194.152437,2017.372383,1887.387049,1927.942473,1968.497897,1902.985289,1800.036905,...,1871.788809,2006.973556,2051.688511,1888.426932,2020.492031,1983.056255,1806.276201,2086.004639,2103.682645,2013.212852
2018,1936.961910,1941.125181,1850.574034,1851.614851,1935.921092,1961.941537,1944.247635,1892.206745,1852.655669,1981.717075,...,2148.247922,1832.880131,2316.860404,2005.655885,2067.064134,1983.798711,2010.859973,1984.839529,2060.819227,1969.227262
2019,1968.344759,1917.588061,1834.347076,1972.405295,1961.238822,2051.585745,1921.648597,1965.299358,1913.527525,1831.301674,...,2047.525209,1926.724267,2255.627672,2033.313333,2103.357577,1989.662573,1992.707975,1906.421588,1921.648597,2009.965252
2020,2060.000000,2116.000000,2060.000000,1987.000000,2033.000000,1906.000000,1959.000000,1818.000000,1855.000000,2017.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [197]:
#hide
expected = amended_deaths.loc[2015:2019].mean()
ci = amended_deaths.loc[2015:2019].std(ddof=1)

In [198]:
#hide
df_expected = pd.DataFrame()
df_expected["Expected"] = expected
df_expected["lower"] = expected - ci
df_expected["upper"] = expected + ci
df_expected["lower 2"] = expected - 2*ci
df_expected["upper 2"] = expected + 2*ci

In [199]:
#hide_input
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

base = alt.Chart(amended_deaths.reset_index().melt("index", value_name="defunciones").rename(columns={"index":"año"})).mark_line(point=True).encode(
    x = alt.X("semana:Q",scale=alt.Scale(domain=(1, 52))),
    y = alt.Y("defunciones", scale=alt.Scale(domain=(0,3500))),
    #size=alt.Size('año:N', legend=None),
    color=alt.Color('año:N', scale=alt.Scale(range=['lightgray','lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'lightgray', 'red'], 
                                             domain=["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]))
)

alt.layer(
    base, # base line chart
    
    alt.Chart(df_expected.reset_index()).mark_area(
        opacity=0.5
    ).encode(
        x="semana",
        y="lower:Q",
        y2="upper:Q"
    ),
    
    alt.Chart(df_expected.reset_index()).mark_area(
        opacity=0.3
    ).encode(
        x="semana",
        y="lower 2:Q",
        y2="upper 2:Q"
    ),
    
    alt.Chart(expected.reset_index().rename(columns={0:"defunciones"})).mark_line(point=True).encode(
    x = "semana",
    y = "defunciones"
    ),
    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x = alt.X('semana:Q', scale=alt.Scale(domain=(1, 52)), axis=alt.Axis(title='Semanas'), sort=None)
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='defunciones'
    ).transform_filter(label),

    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='defunciones'
    ).transform_filter(label),
    
    data=amended_deaths.reset_index().melt("index", value_name="defunciones").rename(columns={"index":"año"})
).properties(
    title = f'Defunciones inscritas en Chile por semana (corregido) hasta semana {current_week-1}',
    width=600
)

alt.LayerChart(...)

In [93]:
#hide
amended_deaths.index

Int64Index([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020], dtype='int64')

In [157]:
#hide
(amended_deaths.loc[2020,:12] - expected).sum()

481.5865043884494

In [158]:
#hide
excess_mortality = []
for i in np.arange(1,23):
    excess_mortality.append((amended_deaths.loc[2020,:i] - expected).sum())

In [159]:
#hide
excess_mortality = pd.Series(excess_mortality)
excess_mortality

0       50.850209
1      202.276298
2      306.947977
3      306.833536
4      359.109322
5      307.906842
6      346.359991
7      214.551868
8      142.962653
9      293.951568
10     371.408951
11     481.586504
12     500.727450
13     575.815546
14     511.002510
15     710.043881
16     710.516165
17     643.175312
18     957.678329
19    1230.835283
20    1728.762516
21    2402.914617
dtype: float64

In [161]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [162]:
#hide
casos_raw.loc["Metropolitana"].head(2)

2020-03-03    0
2020-03-04    1
Name: Metropolitana, dtype: int64

In [163]:
#hide
_, week_first_case, _ = pd.to_datetime("2020-03-03").isocalendar()
week_first_case

10

In [184]:
#hide
deaths_raw = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv")

In [165]:
#hide
deaths_raw[deaths_raw["Fallecidos"]>0].head(2)

,Fecha,Casos nuevos con sintomas,Casos totales,Casos recuperados,Fallecidos,Casos activos,Casos nuevos sin sintomas,Casos nuevos totales
19,2020-03-22,95.0,632.0,8.0,1.0,623.0,NaN,95.0
20,2020-03-23,114.0,746.0,11.0,1.0,734.0,NaN,114.0


In [166]:
#hide
_, week_first_death, _ = pd.to_datetime("2020-03-22").isocalendar()
week_first_death

12

In [167]:
#hide_input
display(Markdown(f"Entre las semanas 12 y {current_week-1}, hubo un total de {amended_deaths.loc[2020,12:22].sum():.0f} defunciones inscritas."))
display(Markdown(f"El número esperado de defunciones inscritas entre esas semanas es de {expected.loc[12:22].sum():.0f}, para una tase de crecimiento de defunciones inscritas entre 2019/2020 de {100*(gr_mean-1):.2f}%"))
display(Markdown(f"Exceso de defunciones inscritas: {(amended_deaths.loc[2020,12:22].sum()-expected.loc[12:22].sum()):.0f}"))

Entre las semanas 12 y 22, hubo un total de 24554 defunciones inscritas.

El número esperado de defunciones inscritas entre esas semanas es de 22522, para una tase de crecimiento de defunciones inscritas entre 2019/2020 de 1.51%

Exceso de defunciones inscritas: 2032

In [168]:
#hide
totales_covid19 = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv",
    index_col="Fecha")

In [169]:
#hide
totales_covid19.head(2)

,Casos nuevos con sintomas,Casos totales,Casos recuperados,Fallecidos,Casos activos,Casos nuevos sin sintomas,Casos nuevos totales
Fecha,,,,,,,
2020-03-03,0.0,1.0,0.0,0.0,1.0,NaN,0.0
2020-03-04,2.0,3.0,0.0,0.0,3.0,NaN,2.0


In [170]:
#hide
_, week_first_case, _ = pd.to_datetime("2020-03-03").isocalendar()
week_first_case

10

In [171]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [172]:
#hide
casos_raw.loc["Metropolitana"].head(2)

2020-03-03    0
2020-03-04    1
Name: Metropolitana, dtype: int64

In [175]:
#hide
deaths_raw = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv")

In [176]:
#hide
deaths_raw[deaths_raw["Fallecidos"]>0].head(2)

,Fecha,Casos nuevos con sintomas,Casos totales,Casos recuperados,Fallecidos,Casos activos,Casos nuevos sin sintomas,Casos nuevos totales
19,2020-03-22,95.0,632.0,8.0,1.0,623.0,NaN,95.0
20,2020-03-23,114.0,746.0,11.0,1.0,734.0,NaN,114.0


In [142]:
#hide
_, week_first_death, _ = pd.to_datetime("2020-03-22").isocalendar()
week_first_death

12

In [177]:
#hide
data = pd.DataFrame()
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-03-24-CasosConfirmados-totalRegional.csv",
    index_col='Region')
data['2020-03-24'] = data_raw['Fallecidos']

In [178]:
#hide
first_death_date = '2020-03-24'
total_days = (pd.to_datetime(today)-pd.to_datetime(first_death_date)).days

In [179]:
#hide
pd.to_datetime("2020-06-01").isocalendar()

(2020, 23, 1)

In [181]:
#hide
# s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" + date + "-CasosConfirmados-totalRegional.csv"
# data_by_date = pd.read_csv(s)
# idx_fallecidos = [i for i, x in enumerate(data_by_date.columns.str.contains("Fallecidos", case=False)) if x]

In [182]:
#hide
for i in np.arange(total_days+1):
    date = (pd.to_datetime(first_death_date)+pd.DateOffset(i)).strftime('%Y-%m-%d')
    s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" + date + "-CasosConfirmados-totalRegional.csv"
    data_by_date = pd.read_csv(s)
    if 'Fallecidos' in data_by_date.columns:
        data[date] = data_by_date["Fallecidos"].values
    elif 'Casos fallecidos' in data_by_date.columns:
        data[date] = data_by_date["Casos fallecidos"].values
    elif 'Fallecidos totales' in data_by_date.columns:
        data[date] = data_by_date["Fallecidos totales"].values
    elif 'Fallecidos totales ' in data_by_date.columns:
        data[date] = data_by_date["Fallecidos totales "].values
    else:
        data[date] = data_by_date[" Casos fallecidos"].values

In [183]:
#hide
data

,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,...,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04
Region,,,,,,,,,,,,,,,,,,,,,
Arica y Parinacota,0,0,0,0,0,0,0,0,0,0,...,7,7,7,7,7,7,8,8,8,8
Tarapacá,0,0,0,0,0,0,0,0,0,0,...,8,11,12,14,18,21,22,23,28,28
Antofagasta,0,0,0,0,0,0,0,0,0,0,...,23,25,28,28,29,31,32,34,36,38
Atacama,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Coquimbo,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,4,4
Valparaíso,0,0,0,0,0,0,0,1,1,1,...,57,58,60,61,63,66,70,72,77,78
Metropolitana,2,2,3,3,3,3,3,4,5,6,...,567,595,638,685,728,775,824,894,961,1034
O’Higgins,0,0,0,0,0,0,0,0,0,0,...,13,14,14,14,16,18,18,18,20,21
Maule,0,0,0,0,0,0,1,1,1,1,...,15,15,15,15,15,15,17,17,19,21
